In [ ]:
import os 
import numpy as np
import pandas as pd

import openpyxl

import matplotlib.pyplot as plt


import jou_gen as jg

from io import StringIO

import plotly.graph_objects as go

from numpy.polynomial.polynomial import Polynomial

import ansys.fluent.core as pyfluent

In [ ]:
# %reload_ext autoreload

In [ ]:
# %autoreload 2

In [ ]:
folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\19MPE_woch\fluent'

N_junction = 38
N_channels = 304
Area_junction = 44.0414E-6

N_big_it = 25
N_it = 40

In [ ]:
solver_path = folder_path + '\\' + 'server_info-15088.txt'
solver = pyfluent.connect_to_fluent(server_info_filepath=solver_path)
tui = solver.tui

In [ ]:
def process_df(df):
    df['index'] = df['Component'].str.extract(r'distrib_ch_(\d+)')
    df['index'] = df['index'].astype(float)

    mdot_distrib_inlet = df[df['Component'] == 'distrib_inlet']['mdot'].values[0]
    df = df.dropna(subset=['index'])

    positive_count = (df['mdot'] > 0).sum()
    null_count = (df['mdot'] == 0).sum()

    if positive_count > 0:

        perc_to_adjust = ( 1 - mdot_distrib_inlet/-df[df['mdot'] < 0]['mdot'].sum() )
        perc_to_distribute = 0.01
        mdot_to_distribute = perc_to_distribute * -df[df['mdot'] < 0]['mdot'].sum()
        perc_to_remove = perc_to_adjust + perc_to_distribute

        df['mdot_updated'] = np.where(df['mdot'] < 0, (1-perc_to_remove)*df['mdot'], mdot_to_distribute / positive_count)
        df['mdot'] = df['mdot_updated'].abs()

    elif null_count > 0:

        perc_to_distribute = 0.05
        mdot_to_distribute = perc_to_distribute * -df[df['mdot'] < 0]['mdot'].sum()

        df['mdot_updated'] = np.where(df['mdot'] < 0, (1-perc_to_distribute)*df['mdot'], mdot_to_distribute / null_count)
        df['mdot'] = df['mdot_updated'].abs()

    else:
        df['mdot'] = df['mdot'].abs()

    df['mdot_ch_norm'] = df['mdot']/(df['mdot'].sum()/N_junction)
    df['new_input'] = df['mdot_ch_norm'] * (mdot_distrib_inlet/N_junction)

    return df,mdot_distrib_inlet

In [ ]:
def create_bc_input(df):

    string_list = []

    for i in range(N_junction):
        string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {df.loc[i+1,'new_input']} [kg / s] / rho_water ) / (44.0414E-6 [m^2])"))

    jg.concatenate_and_write_to_file(string_list,folder_path+'\\'+'change_bc_v.txt')

Set BC of collector channels inlets at velocity inlet with even flow distribution

In [ ]:
string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"((mdot_distrib_inlet/rho_water)/{N_junction})/({Area_junction} [m^2])"))
jg.concatenate_and_write_to_file(string_list,folder_path + '\\' + 'change_bc_v_uniform.txt')
tui.file.read_journal('change_bc_v_uniform.txt')

Set BC of distributor channels outlets with an offset of pressure and even flow distribution assumption as well

In [ ]:
string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_pressure_outlet(f"distrib_ch_{i+1}",f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( mdot_distrib_inlet / {N_channels})",backflow_direction_specification_method='From Neighboring Cell'))
jg.concatenate_and_write_to_file(string_list,folder_path + '\\' + 'change_bc_po_uniform.txt')
tui.file.read_journal('change_bc_po_uniform.txt')

Initialization from distrib_inlet (39)

In [ ]:
init_surface_id = 39
init_sp = 170

gui_standard_init_cmd = f"""/file/set-tui-version "22.2"
(cx-gui-do cx-set-list-tree-selections "NavigationPane*Frame2*Table1*List_Tree2" (list "Solution|Initialization"))
(cx-gui-do cx-set-list-selections "Solution Initialization*Table1*DropDownList1(Compute from)" '( {init_surface_id}))
(cx-gui-do cx-activate-item "Solution Initialization*Table1*DropDownList1(Compute from)")
(cx-gui-do cx-set-real-entry-list "Solution Initialization*Table1*Table7(Initial Values)*RealEntry1(Gauge Pressure)" '( {init_sp}))
(cx-gui-do cx-activate-item "Solution Initialization*Table1*Table7(Initial Values)*RealEntry1(Gauge Pressure)")
(cx-gui-do cx-activate-item "Solution Initialization*Table1*Frame9*PushButton1(Initialize)")
(cx-gui-do cx-activate-item "Question*OK")
"""

with open(folder_path + '\\' + 'standard_init.txt', "w") as file:
    file.write(gui_standard_init_cmd)

In [ ]:
tui.file.read_journal('standard_init.txt')

Make ~100 iterations

In [ ]:
solver.solution.run_calculation.iterate(number_of_iterations=70)

Then put the BC with non uniform flow distribution

In [ ]:
string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_pressure_outlet(f"distrib_ch_{i+1}",f"{{sp_coll_ch_{i+1}}} - 1411785 [Pa s / kg] * ( {{mdot_distrib_ch_{i+1}}} / 8)",backflow_direction_specification_method='From Neighboring Cell'))
jg.concatenate_and_write_to_file(string_list,folder_path + '\\' + 'change_bc_po_non_uniform_new_version.txt')

In [ ]:
string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_pressure_outlet(f"distrib_ch_{i+1}",f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",backflow_direction_specification_method='From Neighboring Cell'))
jg.concatenate_and_write_to_file(string_list,folder_path + '\\' + 'change_bc_po_non_uniform.txt')

tui.file.read_journal('change_bc_po_non_uniform.txt')

In [ ]:
solver.solution.run_calculation.iterate(number_of_iterations=40)

In [ ]:
# tui.file.read_journal('prevent_reverse_flow_every_pressure_outlet.txt')

HERE SOLVE THE CASE AND DATA SOLVE 100 L/H WITH EVEN FLOW

In [ ]:
solver.solution.run_calculation.iterate(number_of_iterations=20)

Run big iterations

In [ ]:
list_df = []
list_mdot_distrib_inlet = []

In [ ]:
i = 0

print('N_big_it:',i)

tui.file.read_journal('write_mdot_report.txt')
jg.convert_report(folder_path,'mdot_report_temp.txt','mdot',output_file_name_wo_extension=f'mdot_distrib_ch_report_{i}')

In [ ]:
df = pd.read_csv(folder_path+'\\'+f'mdot_distrib_ch_report_{0}.csv', header=0)
df['index'] = df['Component'].str.extract(r'distrib_ch_(\d+)')
df['index'] = df['index'].astype(float)
mdot_distrib_inlet = df[df['Component'] == 'distrib_inlet']['mdot'].values[0]
df = df.dropna(subset=['index'])

df.set_index('index',inplace=True)
df.sort_index(inplace=True)

In [ ]:
positive_count = (df['mdot'] > 0).sum()
null_count = (df['mdot'] == 0).sum()

if positive_count > 0:

    perc_to_adjust = ( 1 - mdot_distrib_inlet/-df.loc[df['mdot'] < 0]['mdot'].sum() )
    perc_to_distribute = 0.01
    mdot_to_distribute = perc_to_distribute * -df.loc[df['mdot'] < 0]['mdot'].sum()
    perc_to_remove = perc_to_adjust + perc_to_distribute

    df['mdot_updated'] = np.where(df['mdot'] < 0, (1-perc_to_remove)*df['mdot'], mdot_to_distribute / positive_count)
    df['mdot'] = df['mdot_updated'].abs()

elif null_count > 0:

    perc_to_distribute = 0.01
    mdot_to_distribute = perc_to_distribute * -df[df['mdot'] < 0]['mdot'].sum()

    df['mdot_updated'] = np.where(df['mdot'] < 0, (1-perc_to_distribute)*df['mdot'], mdot_to_distribute / null_count)
    df['mdot'] = df['mdot_updated'].abs()

else:
    df['mdot'] = df['mdot'].abs()

df['mdot_ch_norm'] = df['mdot']/(df['mdot'].sum()/N_junction)
df['new_input'] = df['mdot_ch_norm'] * (mdot_distrib_inlet/N_junction)

In [ ]:
if i == 0 :
    pass
else:
    df['new_input'] = (list_df[i-1]['new_input'] + df['new_input'])/2

In [ ]:
plt.scatter(df.index,df['new_input']/(mdot_distrib_inlet/38))

In [ ]:
# Prepare the data
x = df.index
y = df['mdot_ch_norm']

In [ ]:
# Fit a polynomial of degree 2
degree = 2
coefs = Polynomial.fit(x, y, degree).convert().coef

# Generate a range of x values for plotting the fit
x_fit = x

param = 0.8

y_fit = np.polyval(param*coefs[::-1], x_fit) + (1-param) # Reverse coefs for np.polyval

# Original data
plt.scatter(x, y, label='Original Data')

# Polynomial fit
plt.plot(x_fit, y_fit, color='red', label='Polynomial fit degree 2')

plt.legend()
plt.show()

In [ ]:
df['new_input'] = y_fit * (mdot_distrib_inlet/N_junction)

In [ ]:
plt.scatter(df.index,df['new_input']/(mdot_distrib_inlet/38))

In [ ]:
df['new_input'].sum()

In [ ]:
list_df.append(df)
list_mdot_distrib_inlet.append(mdot_distrib_inlet)

In [ ]:
string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_pressure_outlet(f"distrib_ch_{i+1}",f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",backflow_direction_specification_method='prevent backflow',target_mass_flow_rate=True,mdot=df.loc[i+1,'new_input'],pressure_upper_bound = 1000.))
jg.concatenate_and_write_to_file(string_list,folder_path + '\\' + 'change_bc_po_non_uniform_target.txt')

In [ ]:
tui.file.read_journal('change_bc_po_non_uniform_target.txt')

In [ ]:
create_bc_input(df)

tui.file.read_journal('change_bc_v.txt')

In [ ]:
solver.solution.run_calculation.iterate(number_of_iterations=50)

In [ ]:
file_path = folder_path + '\\' + 'create_residuals_file.txt'
with open(file_path, 'w') as file:
    file.write(jg.write_residuals_file(f"big_it_{i}_residuals_report.txt"))

tui.file.read_journal('create_residuals_file.txt')

RUN BIG IT

In [ ]:
list_df = []
list_mdot_distrib_inlet = []

for i in range(N_big_it):

    print('N_big_it:',i)

    tui.file.read_journal('write_mdot_report.txt')
    jg.convert_report(folder_path,'mdot_report_temp.txt','mdot',output_file_name_wo_extension=f'mdot_distrib_ch_report_{i}')
    df = pd.read_csv(folder_path+'\\'+f'mdot_distrib_ch_report_{i}.csv', header=0)

    df, mdot_distrib_inlet = process_df(df)
    list_df.append(df)
    list_mdot_distrib_inlet.append(mdot_distrib_inlet)

    if i == 0 :
        pass
    else:
        df['new_input'] = (list_df[i-1]['new_input'] + list_df[i]['new_input'])/2

    create_bc_input(df)

    tui.file.read_journal('change_bc_v.txt')
        
    solver.solution.run_calculation.iterate(number_of_iterations=N_it)

    file_path = folder_path + '\\' + 'create_residuals_file.txt'
    with open(file_path, 'w') as file:
        file.write(jg.write_residuals_file(f"big_it_{i}_residuals_report.txt"))

    tui.file.read_journal('create_residuals_file.txt')

In [ ]:
list_df = []

for i in range(N_big_it):
    df = pd.read_csv(folder_path+'\\'+f'mdot_distrib_ch_report_{i}.csv', header=0)
    df['index'] = df['Component'].str.extract(r'distrib_ch_(\d+)')
    df['index'] = df['index'].astype(float)
    list_df.append(df)

In [ ]:
fig, ax = plt.subplots()

for i in range(4):
    ax.scatter(list_df[i]['index'], -list_df[i]['mdot'], label=f'Norm. mdot distrib at big it. {i}')

# put the legend at right side of the graph
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()